In [31]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_squared_error

In [32]:
games = pd.read_csv(f'data/cumulative_with_cur_year_and_last_year_ratings_2010_2023.csv')
games = games[games['completed'] == True]
# games = games[games['year'] != 2023]
games['last_year_team_rating*num_games_into_season'] = games.apply(lambda row: row['last_year_team_rating'] * row['num_games_into_season'], axis=1)
games['last_year_opponent_rating*num_games_into_season'] = games.apply(lambda row: row['last_year_opponent_rating'] * row['num_games_into_season'], axis=1)

In [10]:

def backtest(model, games, x_features, label):
    num_games_increment = 10
    errors = []
    min_num_games = games['num_games_into_season'].min()
    max_num_games = games['num_games_into_season'].max()

    for num_games in range(min_num_games, max_num_games, num_games_increment):
        print(num_games)
        train = games[games['num_games_into_season'] < num_games]
        if len(train) == 0:
            continue
        test = games[num_games + num_games_increment > games['num_games_into_season'] >= num_games]
        X = train[x_features]
        y = train[label]
        model.fit(X, y)
        X_test = test[x_features]
        y_test = test[label]
        y_pred = model.predict(X_test)
        test.loc[:,'pred_margin'] = y_pred
        test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2
        errors += list(test['mse'])

        print('Num games: ', num_games, 'MSE on sample: ', round(mean_squared_error(y_test, y_pred), 2), 'MSE on all games: ', round(np.mean(errors), 2))
    return errors

params = {'max_depth': 5, 'learning_rate': 0.014567791445364069, 'n_estimators': 655, 'min_child_weight': 2, 'gamma': 0.11052774751544212, 'subsample': 0.9899289938848144, 'colsample_bytree': 0.9249071617042357, 'reg_alpha': 0.4468005337539522, 'reg_lambda': 0.24513091931966713, 'random_state': 996}
model = XGBRegressor(**params)

x_features = ['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']
label = 'margin'
errors = backtest(model, games, x_features, label)







0
2


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  2 MSE on sample:  333.48 MSE on all games:  333.48
14


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  14 MSE on sample:  54.02 MSE on all games:  293.56
16


...

/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  43 MSE on sample:  254.49 MSE on all games:  267.76
51


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/2412559932.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  51 MSE on sample:  169.56 MSE on all games:  260.34
55


KeyboardInterrupt: 

In [57]:
X_train, X_test, y_train, y_test = train_test_split(games[['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'last_year_team_rating*num_games_into_season', 'last_year_opponent_rating*num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']], games['margin'], test_size=0.2, random_state=41)

In [17]:
# use optuna to tune the parameters
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=100)

In [ ]:
print('Best parameters', study.best_params)
print('Best value', study.best_value)

In [3]:
games['win'] = games.apply(lambda row: 1 if row['margin'] > 0 else 0, axis=1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(games[['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'last_year_team_rating*num_games_into_season', 'last_year_opponent_rating*num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']], games['win'], test_size=0.2, random_state=41)

In [60]:
# use optuna to tune the parameters for a classifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = XGBClassifier(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)
    return log_loss(y_test, y_pred)

In [ ]:
study = optuna.create_study(direction='minimize', study_name='win_probability')
study.optimize(objective, n_trials=100)

In [32]:
y_pred = [0.5] * len(y_test)
print('Baseline log loss', log_loss(y_test, y_pred))

Baseline log loss 0.6931471805599452


In [9]:
games = pd.read_csv(f'data/cumulative_with_cur_year_and_last_year_ratings_2010_2023.csv')
games = games[games['completed'] == True]
games = games[games['year'] == 2023]
games['win'] = games.apply(lambda row: 1 if row['margin'] > 0 else 0, axis=1)
# test the model on the 2023 season
params = {'max_depth': 1, 'learning_rate': 0.6642774658597874, 'n_estimators': 365, 'min_child_weight': 3, 'gamma': 0.9138018877462927, 'subsample': 0.9850665433158399, 'colsample_bytree': 0.6714438416337997, 'reg_alpha': 0.6338885144969151, 'reg_lambda': 0.18021132749565166, 'random_state': 23}
model = XGBClassifier(**params)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)
print('Log loss', log_loss(y_test, y_pred))

NameError: name 'X_train' is not defined

In [36]:

from sklearn.metrics import log_loss

def backtest(model, games, x_features, label):
    errors = []
    for num_games in sorted(games['num_games_into_season'].unique()):
        train = games[games['num_games_into_season'] < num_games]
        if len(train) == 0:
            continue
        test = games[games['num_games_into_season'] == num_games]
        X = train[x_features]
        y = train[label]
        model.fit(X, y)
        X_test = test[x_features]
        y_test = test[label]
        y_pred = model.predict_proba(X_test)[::,1]
        # get log loss
        test['pred_prob'] = y_pred
        score = log_loss(test['team_win'], test['pred_prob'], labels=[0, 1])
        errors += [score] * len(test)

        print('Num games: ', num_games, 'log loss on sample: ', score, 'log loss overall: ', np.mean(errors))
    return errors

games['team_win'] = games.apply(lambda row: 1 if row['margin'] > 0 else 0, axis=1)
params = {'max_depth': 5, 'learning_rate': 0.01337501236333186, 'n_estimators': 615, 'min_child_weight': 6, 'gamma': 0.22171810700204012, 'subsample': 0.23183800840898533, 'colsample_bytree': 0.29826505641378537, 'reg_alpha': 0.5869931848470185, 'reg_lambda': 0.01392437600344064, 'random_state': 931}
model = XGBClassifier(**params)

x_features = ['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']
label = 'team_win'
errors = backtest(model, games, x_features, label)

/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/48059706.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  2 log loss on sample:  0.6931471824645996 log loss overall:  0.6931471824645996


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/48059706.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  3 log loss on sample:  0.6622084304690361 log loss overall:  0.6772358242954527


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/48059706.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


...

/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/48059706.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  640 log loss on sample:  0.6488781372706095 log loss overall:  0.6294622557155909


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/48059706.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  641 log loss on sample:  0.6930199973285198 log loss overall:  0.6294925935397498


KeyboardInterrupt: 